## Non-stationary & 確率的方策について，制約付きMDPをDPで解く

参考
* [Dynamic programming equations for discounted constrained stochastic control](https://ieeexplore.ieee.org/abstract/document/1298995)
* [Non-randomized control of constrained Markov decision processes](https://ieeexplore.ieee.org/document/1656446)

[RL_CMDP_by_DP.ipynb](RL_CMDP_by_DP.ipynb)の続き．
Non-stationaryな方策を考えると，下界ではなく厳密な解を得ることができます．（こっちのほうが丁寧な議論かもしれません）
また，もとの論文は履歴依存の方策ですが，今回はnon-stationary&確率的なマルコフ方策でやってみます（こっちの方が汎用性高そうです）．

表記
* MDP：$(S, A, P)$．（元論文では状態$x$で選択可能な行動$A(x)\subset A$が定義されてますが，面倒なので$A(x)=A\; \forall x \in S$とします）
* 全てのNステップまでのNon-stationaryなマルコフ方策：
$$\Pi_N=\{\{\pi_0 : S \to \Delta(A), \pi_1:S \to \Delta(A), \dots, \pi_{N} : S \to \Delta(A) \}\mid \pi_k (\cdot \mid s_k) \in \Delta(A)\; \text{ for all } \;s_k \in S, k \geq 0\}$$
* $\pi_{0:k}$で，$0$から$k$ステップ目までを含む方策の系列とする．
* $c(x, u)$はコスト関数, $d(x, u)$は制約関数
* 目的関数：

$$
J_N^\pi(s_0) = E\left[\sum_{j=0}^{N} \gamma^j c(s_j, a_j) \mid \pi \right]
$$

$$
H_N^\pi(s_0) = E\left[\sum_{j=0}^{N} \gamma^j d(s_j, a_j)\mid \pi \right]
$$

* 状態$x$について，制約を満たすような方策が存在する閾値の集合：
    * $\Phi_N(s) = \{\kappa \mid \exist \pi \in \Pi \text{ such that } H^\pi_N(s) \leq \kappa \}$
    * これは$\Phi_N(s)=\left[H_N(s), \infty\right)$ where $H_N(s)=\inf _{\pi \in \Pi} H_N^\pi(s)$であることに注意．
* $\beta: S \rightarrow \mathbb{R}$：各状態に割り当てる制約の閾値の関数．$\beta(x) \in \Phi_N(x)$ $\forall x \in S$とする．

---

**問題設定の目的（有限ホライゾン）**

ゴール：ある固定された状態$s_0$で制約を満たし，かつ収益を最大化させる方策が欲しい．つまり，次を満たす$\pi^*_{N,\beta}$を見つけたい．

$$
\Pi^\text{safe}_N(s_0) = \left\{\pi \in \Pi \mid H_N^\pi(s_0) \leq \beta(s_0)\right\}
$$

として，

$$
J_N^{\pi_{N, \beta}^*}(s_0)=\inf_{\pi \in \Pi_N^\text{safe}(s_0)} J^\pi_N(s_0)
$$

このような方策$\pi^*_{N,\beta}$を見つけることを考えよう．

**注意**

これは特定の初期状態$x_0$についてのみの話であるが，初期状態上の分布を考えても問題ない．
例えば普通のMDPについて，新しい初期状態$x_0$を追加し，そこから全ての状態に初期状態の分布に従って遷移するようにすれば良い．

---

## ベルマン作用素

ある状態$s$で，制約$\kappa$のもとで，達成される最適な価値を定義しよう．
* $\kappa \in \Phi_N(s)$の場合，つまり状態$s$について，$\kappa$を達成する方策が存在すれば，

$$
V_N(s, \kappa)= \inf_{\{\pi \in \Pi_N \mid H^\pi_N(s) \leq \kappa\}} J^\pi_N(s)
$$

* $\kappa \notin \Phi_N(s)$の場合，つまり状態$s$について$\kappa$を達成する方策が無い場合は，
$$V_N(s, \kappa)= \bar{C}$$
とする．

---

次の関数 $F_N: S \times \mathbb{R} \rightarrow 2^{\Delta(A) \times B(S)}$ を考えよう．つまり，状態と閾値を受け取って，分布と制約関数の集合を返す関数である．

$$
\begin{aligned}
& F_N(s, \kappa)=\left\{\left(g, \beta^{\prime}\right) \mid g \in \Delta(A), \beta^{\prime}\left(s^{\prime}\right) \in \Phi_N\left(s^{\prime}\right)\right.
\text { for all } s^{\prime} \in S,\\
&\left. \text { and } \sum_{a\in A}g(a) \left(d(s, a)+\gamma \sum_{s^{\prime} \in S} P\left(s^{\prime} \mid s, a\right)\beta^{\prime}\left(s^{\prime}\right) \right) \leq \kappa\right\} .
\end{aligned}
$$

つまり，与えられた状態と閾値について，閾値を満たすような行動と閾値関数の集合のこと．
このとき，次のベルマン作用素を考えよう．

---

**制約付きベルマン作用素**

次のベルマン作用素を考えよう：

$F_N(s, \kappa) \neq \empty$ならば，

$$
T_{F_N} V(s, \kappa)= 
\quad \inf _{\left(g, \beta^{\prime}\right) \in F_N(s, \kappa)}
\left\{\sum_{a \in A} g(a) \left( c(s, a)+\gamma \sum_{s^{\prime} \in S} P\left(s^{\prime} \mid s, a\right) V\left(s^{\prime}, \beta^{\prime}\left(s^{\prime}\right)\right)\right)\right\}
$$

$F(s, \kappa) = \empty$ならば，

$$T_{F_N} V(s, \kappa)=\bar{C}$$

---

**直感**

$F_N$の中の要素$(g, \beta) \in F_N(s, \kappa)$を考えてみましょう．
この$g, \beta$は

$$
\sum_{a\in A}g(a) \left(d(s, a)+\gamma \sum_{s^{\prime} \in S} P\left(s^{\prime} \mid s, a\right)\beta\left(s^{\prime}\right) \right) \leq \kappa
$$


を全ての$s\in S$について満たす．よって，最初のステップで分布$g$を選択し，次のステップ以降では制約$\beta$を達成するような方策は，全体で制約$\kappa$を満たす．

このような$g$と$\beta$のうち，最も良いものを選び続ければ，全体で$\kappa$を満たしながらコストの最小化が達成できそう．

![CMDP-DP](figs/CMDP_DP.jpg)



---

**ベルマン方程式**

上で定義した$V_N$について，次が成立する．

$$
V_{N+1}(s, \kappa)=T_{F_N} V_N(s, \kappa)
$$

つまり，問題設定の目的である最適方策の価値をベルマン作用素に入れても，同じ価値が返ってくる．

**証明**

適当な$(s_0, \kappa_0)$について考えよう．

まず，$\kappa_0 \notin \Phi_{N+1}\left(s_0\right)$のとき，
$\kappa_0$を達成可能な方策が$s_0$では存在しない．一方で，

$$
\begin{aligned}
& F_N(s_0, \kappa_0)=\left\{\left(g, \beta^{\prime}\right) \mid g \in \Delta(A), \beta^{\prime}\left(s^{\prime}\right) \in \Phi_N\left(s^{\prime}\right)\right.
\text { for all } s^{\prime} \in S,\\
&\left. \text { and } \sum_{a\in A}g(a) \left(d(s_0, a)+\gamma \sum_{s^{\prime} \in S} P\left(s^{\prime} \mid s_0, a\right)\beta^{\prime}\left(s^{\prime}\right) \right) \leq \kappa_0\right\} .
\end{aligned}
$$

であるが，このような$F_N(s_0, \kappa_0)$が空でない場合，$s_0$で$g$, その後で制約$\beta'(s')$を達成する方策が存在してしまうので，$\kappa_0$を達成可能な方策が$s_0$に存在してしまう．よって，$F_N(s_0, \kappa_0)$は空である．

よって定義より，

$$
V_{N+1}\left(s_0, \kappa_0\right)=\bar{C}=T_{F_N} V_N\left(s_0, \kappa_0\right)
$$



続いて，$\kappa_0 \in \Phi_{N+1}\left(s_0\right)$の時を考える．
$\beta_0\left(s_0\right)=\kappa_0$なる関数を定義しよう．
定義より，

$$
\begin{aligned}
V_{N+1}\left(s_0, \kappa_0\right) 
&= \inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}} E\left[\sum_{j=0}^{N+1} \gamma^j c\left(s_j, a_j\right) \mid \pi\right]\\
&= \inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}} E\left[c(s_0, a_0) + \sum_{j=1}^{N+1} \gamma^j c\left(s_j, a_j\right) \mid \pi\right]\\
& =\inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}} E\left[c(s_0, a_0) + \underbrace{E\left[\sum_{j=1}^{N+1} \gamma^j c\left(s_j, a_j\right) \mid \pi_{1:N+1}, S_1=s_1\right]}_{\text{制約違反は}H^{\pi_{1:N+1}}_{N}(s_1)\} \text{以下}} \mid \pi \right] \\
& \geq\inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}} E\left[c(s_0, a_0) + \inf_{\{\pi' \in \Pi_{N} \mid H^{\pi'}_{N}(s_1) \leq H^{\pi_{1:N+1}}_{N}(s_1)\}}E\left[\sum_{j=1}^{N+1} \gamma^j c\left(s_j, a_j\right) \mid \pi', S_1=s_1\right] \mid \pi \right] \\
& =\inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}} E\left[c(s_0, a_0) + \gamma V_N(s_1, H^{\pi_{1:N+1}}_{N}(s_1)) \mid \pi \right] \\
&=\inf_{\{\pi \in \Pi_{N+1} \mid H^\pi_{N+1}(s_0) \leq \kappa_0\}}\sum_{a\in A}\pi(a\mid s_0)\left(c(s_0, a) + \gamma \sum_{s_1 \in S} P(s_1 \mid s_0, a)V_N(s_1, \underbrace{H^{\pi_{1:N+1}}_{N}(s_1)}_{(a)})\right)\\
\end{aligned}
$$

ここで，最後の$(a)$の部分について考えてみましょう．
$\inf$を与える$\pi$を考えてみると，$H^{\pi_{1:N+1}}_{N}$について，

$$
\sum_{a\in A}\pi_0(a \mid s_0) \left(d(s_0, a)+\gamma \sum_{s^{\prime} \in S} P\left(s^{\prime} \mid s_0, a\right)H^{\pi_{1:N+1}}_{N}\left(s^{\prime}\right) \right) \leq \kappa_0
$$

を満たします．よって，上の$(\pi_0, \beta') \in F_N(s_0, \kappa_0)$です．

$$
T_{F_N} V_N(s_0, \kappa_0)= 
\quad \inf _{\left(g, \beta^{\prime}\right) \in F_N(s_0, \kappa_0)}
\left\{\sum_{a \in A} g(a) \left( c(s_0, a)+\gamma \sum_{s_1 \in S} P\left(s_1 \mid s_0, a\right) V_N\left(s_1, \beta^{\prime}\left(s_1\right)\right)\right)\right\}
$$

であるから，

$$
V_{N+1}(s_0, \kappa_0) \geq T_{F_N}V_{N}(s_0, \kappa_0)
$$

が成立する．

---

一方で，
$f: (S\times \mathbb{R}) \to (\Delta(A) \times B(S))$
として，$f=(f^1, f^2)$とする．

また，
$$
(f^1(s, \kappa), f^2(s, \kappa))
= 
\arg \inf_{(g, \beta') \in F_N(s, \kappa)}
\left\{
\sum_{a\in A} g(a) \left(
c(s, a) + \gamma \sum_{s'\in S}P(s'\mid s, a) V_N(s', \beta'(s'))
\right)
\right\}
$$

とする．これを使って，$\beta_0\left(s_0\right)=\kappa_0$なので，

$$
\hat{\pi}_{0} = f^1(s_0, \beta_0(s_0)) = f^1(s_0, \kappa_0)
$$
および
$$
\hat{\pi}_{1:N+1} = \pi^*_{N, f^2(s_0, \beta_0(s_0))} = \pi^*_{N, f^2(s_0, \kappa_0)}
$$
なる方策$\hat{\pi}$を作る．
つまり，
$\hat{\pi}_{1:N+1} = \pi^*_{N, f^2(s_0, \kappa_0)}$は制約$\kappa_0$を満たす方策のうち，コストを最小化する最適なやつ．

このとき，

$$
\begin{aligned}
V_{N+1}(s_0, \kappa_0)
&\underbrace{\leq J_{N+1}^{\hat{\pi}}(s_0)}_{\text{最初の行動は} V_{N+1} \text{より悪いので}}\\
&=\sum_{a\in A}\hat{\pi}_0(a\mid s_0)\left(c(s_0, a) + \gamma \sum_{s_1 \in S} P(s_1 \mid s_0, a)J_N^{\hat{\pi}_{1:N+1}}(s_1))\right)\\
&=\sum_{a\in A}\hat{\pi}_0(a\mid s_0)\left(c(s_0, a) + \gamma \sum_{s_1 \in S} P(s_1 \mid s_0, a)V_N(s_1, \kappa_0))\right)\\
&=T_{F_N}V_N(s_0, \kappa_0)
\end{aligned}
$$

よって，前回のものと合わせて

$$V_{N+1}(s_0, \kappa_0)=T_{F_N}V_N(s_0, \kappa_0)$$
